In [1]:
import cv2
import pickle
import math
from statistics import mean,median,mode 

%load_ext Cython

import numpy as np
import argparse
import time
import cv2
import os
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imutils

import numpy as np

from skimage.measure import compare_ssim
from PIL import Image
import time

import scipy
import random
import math

# Image manipulation.

import imagehash

from numpy import load

from PIL import Image

from numpy import asarray
from numpy import save

import pandas as pd
import constants
import helper



In [2]:
# orb = cv2.ORB_create()
# img1=cv2.imread('frames/101.jpg')
# img2=cv2.imread('frames/1082.jpg')
# kp1, des1 = orb.detectAndCompute(img1,None)
# kp2, des2 = orb.detectAndCompute(img2,None)
# bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# matches = bf.match(des1,des2)
# matches = sorted(matches, key = lambda x:x.distance)
# img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches,None, flags=2)
# cv2.imshow('matches',img3)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))





In [4]:

image_1 = cv2.imread('frames/0.jpg')
start=time.time()

old_gray = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, 1000, 0.01, 10)
mask = np.zeros_like(image_1)

    
frame=cv2.imread('frames/100.jpg')
frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)



# calculate optical flow
p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)


# Select good points
good_new = p1[st==1]
good_old = p0[st==1]

count=0
# draw the tracks
for i,(new,old) in enumerate(zip(good_new,good_old)):
    a,b = new.ravel()
    c,d = old.ravel()
    frame = cv2.circle(frame,(a,b),5,255,-1)
    if abs(d-b)>1 or abs(a-c)>1:
        count = count +1  
        mask = cv2.line(mask, (a,b),(c,d), (0,255,0), 2)
        cv2.circle(mask,(a,b),3,(0,0,255),-1)
        cv2.circle(mask,(c,d),3,(255,0,0),-1)
        img = cv2.add(image_1,mask)
        
cv2.imwrite('flow.jpg',img)
# cv2.imshow('flow',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
    
print(time.time()-start)

0.12928104400634766


In [ ]:

print(count)

In [5]:
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))



def get_flow_count_lucas(image_1,image_2):
    image_1 = cv2.imread(image_1)
    old_gray = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, 1000, 0.01, 10)

    
    image_2 = cv2.imread(image_2)
    frame_gray = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)
    
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)


    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    count=0
    
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        if abs(d-b)>1 or abs(a-c)>1:
            count = count +1     

    return count

In [32]:
df_data = pd.read_csv('IOU_scores_detection_flag_flow_lucas.csv')

In [33]:
df_lucas = pd.DataFrame()

In [34]:
iou_list=[25,50,75,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100]

In [35]:
for i in iou_list:
    print('Copying IOU {}'.format(i))
    df_lucas['image_{}'.format(i)]=df_data['image']
    df_lucas['reference_image_{}'.format(i)]=df_data['reference_image_{}'.format(i)]
    df_lucas['{}_IOU'.format(i)]=df_data['{}_IOU'.format(i)]
    df_lucas['{}_IOU_Flag'.format(i)]=df_data['{}_IOU_Flag'.format(i)]
    
    print('Starting lucas flow for IOU {}%'.format(i))
    temp_df = df_lucas[['image_{}'.format(i),'reference_image_{}'.format(i),'{}_IOU_Flag'.format(i)]]
    list_of_flow = []
    for index,row in temp_df.iterrows():
        if row['{}_IOU_Flag'.format(i)]==1:
            image_1 = 'frames/{}.jpg'.format(row['image_{}'.format(i)])
            image_2 = 'frames/{}.jpg'.format(row['reference_image_{}'.format(i)])
            flow_count= get_flow_count_lucas(image_1,image_2)
            list_of_flow.append(flow_count)
            print(image_1,'|',image_2,'|','iou flag',row['{}_IOU_Flag'.format(i)],'|','flow',flow_count)

        if row['{}_IOU_Flag'.format(i)]==0:
            image_1 = 'frames/{}.jpg'.format(row['image_{}'.format(i)])
            image_2 = 'frames/{}.jpg'.format(row['reference_image_{}'.format(i)])
            flow_count= get_flow_count_lucas(image_1,image_2)
            list_of_flow.append(flow_count)
            print(image_1,'|',image_2,'|','iou flag',row['{}_IOU_Flag'.format(i)],'|','flow',flow_count)
            
    df_lucas['{}_lucas_flow_count'.format(i)]=list_of_flow
    print('Done with {}%'.format(i))

Copying IOU 25
Starting lucas flow for IOU 25%
frames/0.jpg | frames/0.jpg | iou flag 0 | flow 0
frames/1.jpg | frames/0.jpg | iou flag 0 | flow 0
frames/2.jpg | frames/0.jpg | iou flag 0 | flow 1
frames/3.jpg | frames/0.jpg | iou flag 0 | flow 5
frames/4.jpg | frames/0.jpg | iou flag 0 | flow 8
frames/5.jpg | frames/0.jpg | iou flag 0 | flow 9
frames/6.jpg | frames/0.jpg | iou flag 0 | flow 11
frames/7.jpg | frames/0.jpg | iou flag 0 | flow 15
frames/8.jpg | frames/0.jpg | iou flag 0 | flow 23
frames/9.jpg | frames/0.jpg | iou flag 0 | flow 23
frames/10.jpg | frames/0.jpg | iou flag 0 | flow 30
frames/11.jpg | frames/0.jpg | iou flag 0 | flow 32
frames/12.jpg | frames/0.jpg | iou flag 0 | flow 37
frames/13.jpg | frames/0.jpg | iou flag 0 | flow 38
frames/14.jpg | frames/0.jpg | iou flag 0 | flow 39
frames/15.jpg | frames/0.jpg | iou flag 0 | flow 36
frames/16.jpg | frames/0.jpg | iou flag 0 | flow 35
frames/17.jpg | frames/0.jpg | iou flag 0 | flow 31
frames/18.jpg | frames/0.jpg | io

frames/154.jpg | frames/117.jpg | iou flag 0 | flow 188
frames/155.jpg | frames/117.jpg | iou flag 0 | flow 188
frames/156.jpg | frames/117.jpg | iou flag 0 | flow 188
frames/157.jpg | frames/117.jpg | iou flag 0 | flow 193
frames/158.jpg | frames/117.jpg | iou flag 0 | flow 192
frames/159.jpg | frames/117.jpg | iou flag 0 | flow 184
frames/160.jpg | frames/117.jpg | iou flag 0 | flow 188
frames/161.jpg | frames/117.jpg | iou flag 0 | flow 177
frames/162.jpg | frames/117.jpg | iou flag 0 | flow 197
frames/163.jpg | frames/117.jpg | iou flag 0 | flow 182
frames/164.jpg | frames/117.jpg | iou flag 0 | flow 180
frames/165.jpg | frames/117.jpg | iou flag 0 | flow 172
frames/166.jpg | frames/117.jpg | iou flag 0 | flow 177
frames/167.jpg | frames/117.jpg | iou flag 0 | flow 182
frames/168.jpg | frames/117.jpg | iou flag 0 | flow 168
frames/169.jpg | frames/117.jpg | iou flag 0 | flow 170
frames/170.jpg | frames/117.jpg | iou flag 0 | flow 157
frames/171.jpg | frames/117.jpg | iou flag 0 | f

frames/302.jpg | frames/275.jpg | iou flag 0 | flow 64
frames/303.jpg | frames/275.jpg | iou flag 0 | flow 61
frames/304.jpg | frames/275.jpg | iou flag 0 | flow 64
frames/305.jpg | frames/275.jpg | iou flag 0 | flow 60
frames/306.jpg | frames/275.jpg | iou flag 0 | flow 56
frames/307.jpg | frames/275.jpg | iou flag 0 | flow 54
frames/308.jpg | frames/275.jpg | iou flag 0 | flow 56
frames/309.jpg | frames/275.jpg | iou flag 0 | flow 57
frames/310.jpg | frames/275.jpg | iou flag 0 | flow 57
frames/311.jpg | frames/275.jpg | iou flag 0 | flow 58
frames/312.jpg | frames/275.jpg | iou flag 0 | flow 58
frames/313.jpg | frames/275.jpg | iou flag 0 | flow 57
frames/314.jpg | frames/275.jpg | iou flag 0 | flow 59
frames/315.jpg | frames/275.jpg | iou flag 0 | flow 59
frames/316.jpg | frames/275.jpg | iou flag 0 | flow 56
frames/317.jpg | frames/275.jpg | iou flag 0 | flow 53
frames/318.jpg | frames/275.jpg | iou flag 0 | flow 58
frames/319.jpg | frames/275.jpg | iou flag 0 | flow 58
frames/320

frames/454.jpg | frames/275.jpg | iou flag 0 | flow 43
frames/455.jpg | frames/275.jpg | iou flag 0 | flow 45
frames/456.jpg | frames/275.jpg | iou flag 0 | flow 39
frames/457.jpg | frames/275.jpg | iou flag 0 | flow 39
frames/458.jpg | frames/275.jpg | iou flag 0 | flow 38
frames/459.jpg | frames/275.jpg | iou flag 0 | flow 38
frames/460.jpg | frames/275.jpg | iou flag 0 | flow 39
frames/461.jpg | frames/275.jpg | iou flag 0 | flow 39
frames/462.jpg | frames/275.jpg | iou flag 0 | flow 43
frames/463.jpg | frames/275.jpg | iou flag 0 | flow 41
frames/464.jpg | frames/275.jpg | iou flag 0 | flow 39
frames/465.jpg | frames/275.jpg | iou flag 0 | flow 40
frames/466.jpg | frames/275.jpg | iou flag 0 | flow 39
frames/467.jpg | frames/275.jpg | iou flag 0 | flow 40
frames/468.jpg | frames/275.jpg | iou flag 0 | flow 40
frames/469.jpg | frames/275.jpg | iou flag 0 | flow 47
frames/470.jpg | frames/275.jpg | iou flag 0 | flow 51
frames/471.jpg | frames/275.jpg | iou flag 0 | flow 47
frames/472

frames/603.jpg | frames/275.jpg | iou flag 0 | flow 72
frames/604.jpg | frames/275.jpg | iou flag 0 | flow 69
frames/605.jpg | frames/275.jpg | iou flag 0 | flow 69
frames/606.jpg | frames/275.jpg | iou flag 0 | flow 70
frames/607.jpg | frames/275.jpg | iou flag 0 | flow 71
frames/608.jpg | frames/275.jpg | iou flag 0 | flow 75
frames/609.jpg | frames/275.jpg | iou flag 0 | flow 77
frames/610.jpg | frames/275.jpg | iou flag 0 | flow 73
frames/611.jpg | frames/275.jpg | iou flag 0 | flow 71
frames/612.jpg | frames/275.jpg | iou flag 0 | flow 74
frames/613.jpg | frames/275.jpg | iou flag 0 | flow 73
frames/614.jpg | frames/275.jpg | iou flag 0 | flow 73
frames/615.jpg | frames/275.jpg | iou flag 0 | flow 79
frames/616.jpg | frames/275.jpg | iou flag 0 | flow 81
frames/617.jpg | frames/275.jpg | iou flag 0 | flow 83
frames/618.jpg | frames/275.jpg | iou flag 0 | flow 82
frames/619.jpg | frames/275.jpg | iou flag 0 | flow 84
frames/620.jpg | frames/275.jpg | iou flag 0 | flow 84
frames/621

...

frames/335.jpg | frames/334.jpg | iou flag 1 | flow 13
frames/336.jpg | frames/335.jpg | iou flag 1 | flow 14
frames/337.jpg | frames/336.jpg | iou flag 1 | flow 13
frames/338.jpg | frames/337.jpg | iou flag 1 | flow 18
frames/339.jpg | frames/338.jpg | iou flag 1 | flow 14
frames/340.jpg | frames/339.jpg | iou flag 1 | flow 16
frames/341.jpg | frames/340.jpg | iou flag 1 | flow 9
frames/342.jpg | frames/341.jpg | iou flag 1 | flow 9
frames/343.jpg | frames/342.jpg | iou flag 1 | flow 8
frames/344.jpg | frames/343.jpg | iou flag 1 | flow 8
frames/345.jpg | frames/344.jpg | iou flag 1 | flow 12
frames/346.jpg | frames/345.jpg | iou flag 1 | flow 17
frames/347.jpg | frames/346.jpg | iou flag 1 | flow 17
frames/348.jpg | frames/347.jpg | iou flag 1 | flow 16
frames/349.jpg | frames/348.jpg | iou flag 1 | flow 20
frames/350.jpg | frames/349.jpg | iou flag 1 | flow 19
frames/351.jpg | frames/350.jpg | iou flag 1 | flow 18
frames/352.jpg | frames/351.jpg | iou flag 1 | flow 12
frames/353.jpg

frames/487.jpg | frames/486.jpg | iou flag 1 | flow 39
frames/488.jpg | frames/487.jpg | iou flag 1 | flow 37
frames/489.jpg | frames/488.jpg | iou flag 1 | flow 42
frames/490.jpg | frames/489.jpg | iou flag 1 | flow 41
frames/491.jpg | frames/490.jpg | iou flag 1 | flow 41
frames/492.jpg | frames/491.jpg | iou flag 1 | flow 41
frames/493.jpg | frames/492.jpg | iou flag 1 | flow 37
frames/494.jpg | frames/493.jpg | iou flag 1 | flow 31
frames/495.jpg | frames/494.jpg | iou flag 1 | flow 29
frames/496.jpg | frames/495.jpg | iou flag 1 | flow 30
frames/497.jpg | frames/496.jpg | iou flag 1 | flow 31
frames/498.jpg | frames/497.jpg | iou flag 1 | flow 30
frames/499.jpg | frames/498.jpg | iou flag 1 | flow 31
frames/500.jpg | frames/499.jpg | iou flag 1 | flow 30
frames/501.jpg | frames/500.jpg | iou flag 1 | flow 33
frames/502.jpg | frames/501.jpg | iou flag 1 | flow 29
frames/503.jpg | frames/502.jpg | iou flag 1 | flow 29
frames/504.jpg | frames/503.jpg | iou flag 1 | flow 29
frames/505

frames/636.jpg | frames/635.jpg | iou flag 1 | flow 36
frames/637.jpg | frames/636.jpg | iou flag 1 | flow 35
frames/638.jpg | frames/637.jpg | iou flag 1 | flow 40
frames/639.jpg | frames/638.jpg | iou flag 1 | flow 32
frames/640.jpg | frames/639.jpg | iou flag 1 | flow 35
frames/641.jpg | frames/640.jpg | iou flag 1 | flow 33
frames/642.jpg | frames/641.jpg | iou flag 1 | flow 33
frames/643.jpg | frames/642.jpg | iou flag 1 | flow 30
frames/644.jpg | frames/643.jpg | iou flag 1 | flow 29
frames/645.jpg | frames/644.jpg | iou flag 1 | flow 28
frames/646.jpg | frames/645.jpg | iou flag 1 | flow 29
frames/647.jpg | frames/646.jpg | iou flag 1 | flow 26
frames/648.jpg | frames/647.jpg | iou flag 1 | flow 27
frames/649.jpg | frames/648.jpg | iou flag 1 | flow 29
frames/650.jpg | frames/649.jpg | iou flag 1 | flow 26
frames/651.jpg | frames/650.jpg | iou flag 1 | flow 24
frames/652.jpg | frames/651.jpg | iou flag 1 | flow 21
frames/653.jpg | frames/652.jpg | iou flag 1 | flow 20
frames/654

frames/790.jpg | frames/789.jpg | iou flag 1 | flow 34
frames/791.jpg | frames/790.jpg | iou flag 1 | flow 36
frames/792.jpg | frames/791.jpg | iou flag 1 | flow 46
frames/793.jpg | frames/792.jpg | iou flag 1 | flow 37
frames/794.jpg | frames/793.jpg | iou flag 1 | flow 34
frames/795.jpg | frames/794.jpg | iou flag 1 | flow 38
frames/796.jpg | frames/795.jpg | iou flag 1 | flow 36
frames/797.jpg | frames/796.jpg | iou flag 1 | flow 35
frames/798.jpg | frames/797.jpg | iou flag 1 | flow 38
frames/799.jpg | frames/798.jpg | iou flag 1 | flow 37
frames/800.jpg | frames/799.jpg | iou flag 1 | flow 36
frames/801.jpg | frames/800.jpg | iou flag 1 | flow 34
frames/802.jpg | frames/801.jpg | iou flag 1 | flow 38
frames/803.jpg | frames/802.jpg | iou flag 1 | flow 25
frames/804.jpg | frames/803.jpg | iou flag 1 | flow 39
frames/805.jpg | frames/804.jpg | iou flag 1 | flow 36
frames/806.jpg | frames/805.jpg | iou flag 1 | flow 40
frames/807.jpg | frames/806.jpg | iou flag 1 | flow 34
frames/808

frames/939.jpg | frames/938.jpg | iou flag 1 | flow 25
frames/940.jpg | frames/939.jpg | iou flag 1 | flow 22
frames/941.jpg | frames/940.jpg | iou flag 1 | flow 27
frames/942.jpg | frames/941.jpg | iou flag 1 | flow 29
frames/943.jpg | frames/942.jpg | iou flag 1 | flow 15
frames/944.jpg | frames/943.jpg | iou flag 1 | flow 34
frames/945.jpg | frames/944.jpg | iou flag 1 | flow 16
frames/946.jpg | frames/945.jpg | iou flag 1 | flow 30
frames/947.jpg | frames/946.jpg | iou flag 1 | flow 27
frames/948.jpg | frames/947.jpg | iou flag 1 | flow 26
frames/949.jpg | frames/948.jpg | iou flag 1 | flow 19
frames/950.jpg | frames/949.jpg | iou flag 1 | flow 24
frames/951.jpg | frames/950.jpg | iou flag 1 | flow 18
frames/952.jpg | frames/951.jpg | iou flag 1 | flow 16
frames/953.jpg | frames/952.jpg | iou flag 1 | flow 15
frames/954.jpg | frames/953.jpg | iou flag 1 | flow 7
frames/955.jpg | frames/954.jpg | iou flag 1 | flow 26
frames/956.jpg | frames/955.jpg | iou flag 1 | flow 27
frames/957.

In [36]:
df_lucas.to_csv('lucasx.csv',index=False)

In [21]:
df_lucas=pd.read_csv('lucasx.csv')

In [18]:
def divide_chunks(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n]
    
def compare_vectors_using_sliding_window(detection_vector_iou,detection_vector_flow_count):    
    
    matches=0
    bottom_left=0
    top_right = 0
    bottom_right=0
    result_vector_iou=[]
    result_vector_flow_count=[]
    
    for i in range(0,len(detection_vector_iou)):
        if sum(detection_vector_iou[i])>0:
            result_vector_iou.append(1)
        else:                
            result_vector_iou.append(0)
            
        if sum(detection_vector_flow_count[i])>0:
            result_vector_flow_count.append(1)
        else:
            result_vector_flow_count.append(0)
        

        if sum(detection_vector_iou[i])>0 and sum(detection_vector_flow_count[i])>0:
            matches=matches+1
            
        elif sum(detection_vector_iou[i])==0 and sum(detection_vector_flow_count[i])>0:
            bottom_left=bottom_left+1
            
        if sum(detection_vector_iou[i])>0 and sum(detection_vector_flow_count[i])==0:
            top_right = top_right +1
        
        if sum(detection_vector_iou[i])==0 and sum(detection_vector_flow_count[i])==0:
            bottom_right = bottom_right+1
            
#    print(len(result_vector_iou),len(result_vector_ssim))
    #print(matches)
    
    return result_vector_iou,result_vector_flow_count,matches, bottom_left,top_right,bottom_right
    

In [38]:
dp=[[-1 for i in range(0,1083)] for i in range(0,1083)]

In [19]:
def get_results(iou_threshold,percentiles_list):
    print('Working with IOU: ',iou_threshold )
    flow_count_list_for_iou_threshold=[]
    
    temp_df = df_lucas[['{}_IOU_Flag'.format(iou_threshold),'{}_lucas_flow_count'.format(iou_threshold)]].copy()
    
    for index,row in temp_df.iterrows():
        if int(row['{}_IOU_Flag'.format(iou_threshold)])==1:
            flow_count_list_for_iou_threshold.append(row['{}_lucas_flow_count'.format(iou_threshold)])
                
    #print(len(flow_count_list_for_iou_threshold))
    
    matches_per_percentile=[]
    flow_count_values_per_percentile=[]
    number_of_detections_per_percentile=[]
    
    for percentile in percentiles_list:
        #print('IOU: {} | percentile: {}'.format(iou_threshold,percentile))
        
        flow_count_at_percentile = np.percentile(flow_count_list_for_iou_threshold,percentile) 
        #print('max flow count',max(flow_count_list_for_iou_threshold))
        #print('min flow count',min(flow_count_list_for_iou_threshold))
        flow_count_values_per_percentile.append(flow_count_at_percentile)
    
        reference_image = 0
        detection_flag_list_flow_count=[]
        number_of_images=1083
        number_of_detections=0
        #print('Flow Count Threshold', flow_count_at_percentile)
        for i in range(0,1083):
            if dp[reference_image][i]!=-1:
                flow_count= dp[reference_image][i]
            else:
                flow_count=get_flow_count_lucas('frames/{}.jpg'.format(reference_image),'frames/{}.jpg'.format(i))
                dp[reference_image][i]=flow_count
                
            
            if flow_count>flow_count_at_percentile:
                reference_image = i
                number_of_detections=number_of_detections+1
                detection_flag_list_flow_count.append(1)
            else:
                detection_flag_list_flow_count.append(0)
        #print('Dealing with Percentile: ', percentile)
        #print('Flow count threshold', flow_count_at_percentile)
        #print('Number of Detection calls at {}% IOU and {}th percentile: '.format(iou_threshold,percentile), number_of_detections)
        number_of_detections_per_percentile.append(number_of_detections)
        
        iou_partioned=list(divide_chunks(list(df_lucas['{}_IOU_Flag'.format(iou_threshold)]),12))
        #print('iou_partioned_List',len(iou_partioned))
        flow_count_partioned=list(divide_chunks(detection_flag_list_flow_count,12))
        result_vector_iou,result_vector_flow_count,matches,bottom_left,top_right,bottom_right=compare_vectors_using_sliding_window(iou_partioned,flow_count_partioned)
        #print('Window Matches', matches)
        #matches=(matches/91)*100
        print('Percentage Window Matches',matches)
        print('Both agree to call',matches)
        print('Yolo Says dont call, Lucas says call-min',bottom_left)
        print('Yolo says Call, Lucas says dont call-min',top_right)
        print('Yolo says dont call Lucas says dont call-max',bottom_right)
        
        matches_per_percentile.append(matches)
        #print('-------------------')
        
    return matches_per_percentile,number_of_detections_per_percentile,flow_count_values_per_percentile
    #print('----------------')
    #print('##########')
            
        
        
    
    
    

In [36]:

# percentiles_list=[0,5,10,15,16,17,18,19,20,25,50,60,75,80,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100]
# iou_list=[25,50,75,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100]
# percentiles_list=percentiles_list[::-1]
percentiles_list=[50]
iou_list=[100]

rows=[]
for i in iou_list[::-1]:
    #print('Dealing with iou',i)
    temp_row=[]
    temp_row.append(i)
    matches,number_of_detections_per_percentile,flow_count_values_per_percentile=get_results(i,percentiles_list)
    #print('matches',matches)
    #print('no. of detections',number_of_detections_per_percentile)
    #print('flow',flow_count_values_per_percentile)
    temp_row.append(flow_count_values_per_percentile)
    rows.append(temp_row)
    print(temp_row)
    
# dfObjCo = pd.DataFrame(rows) 
# dfObjCo.to_csv('lucas_thresh.csv')

Working with IOU:  100
Percentage Window Matches 84
Both agree to call 84
Yolo Says dont call, Lucas says call-min 0
Yolo says Call, Lucas says dont call-min 7
Yolo says dont call Lucas says dont call-max 0
[100, [28.0]]


In [48]:
rows=[]

print(percentiles_list)
for i in iou_list[::-1]:
    #print('Dealing with iou',i)
    temp_row=[]
    temp_row.append(i)
    matches,number_of_detections_per_percentile,flow_count_values_per_percentile=get_results(i,percentiles_list)
    #print('matches',matches)
    #print('no. of detections',number_of_detections_per_percentile)
    #print('flow',flow_count_values_per_percentile)
    temp_row.append(number_of_detections_per_percentile)
    rows.append(temp_row)
    print(temp_row)
    
dfObj = pd.DataFrame(rows) 
dfObj.to_csv('lucas_function_calls.csv')

[100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 80, 75, 60, 50, 25, 20, 19, 18, 17, 16, 15, 10, 5, 0]
Working with IOU:  100
[100, [60, 86, 101, 114, 131, 147, 160, 175, 186, 197, 211, 222, 237, 253, 260, 265, 318, 367, 513, 609, 867, 926, 926, 936, 936, 936, 951, 988, 1037, 1063]]
Working with IOU:  99
[99, [60, 86, 101, 108, 131, 147, 158, 169, 186, 197, 211, 222, 234, 253, 253, 265, 318, 367, 513, 589, 867, 914, 926, 936, 936, 936, 951, 988, 1037, 1063]]
Working with IOU:  98
[98, [60, 86, 98, 106, 131, 141, 158, 169, 181, 197, 207, 220, 234, 241, 253, 260, 304, 358, 506, 589, 853, 896, 904, 914, 926, 936, 936, 973, 1024, 1063]]
Working with IOU:  97
[97, [60, 86, 95, 106, 125, 141, 146, 169, 175, 186, 207, 220, 222, 237, 253, 253, 287, 345, 506, 572, 832, 875, 896, 896, 904, 914, 926, 951, 1011, 1063]]
Working with IOU:  96
[96, [60, 86, 95, 106, 120, 133, 147, 160, 169, 181, 197, 207, 220, 234, 237, 253, 285, 334, 472, 572, 810, 853, 867, 875, 875, 896, 904, 951,

In [ ]:


rows=[]
print(percentiles_list)
for i in iou_list[::-1]:
    #print('Dealing with iou',i)
    temp_row=[]
    temp_row.append(i)
    matches,number_of_detections_per_percentile,flow_count_values_per_percentile=get_results(i,percentiles_list)
    #print('matches',matches)
    #print('no. of detections',number_of_detections_per_percentile)
    #print('flow',flow_count_values_per_percentile)
    flow_count_values_per_percentile=[int(x) for x in flow_count_values_per_percentile]
    temp_row.append(flow_count_values_per_percentile)
    rows.append(temp_row)
    print(temp_row)

In [2]:
options_v3_default=constants.options_default
net_default=cv2.dnn.readNetFromDarknet(options_v3_default['model'], options_v3_default['load'])
LABELS_default = open(options_v3_default['labelsPath']).read().strip().split("\n")
color = (255, 0, 0)

In [3]:
def predict_object_image(image,net,Labels,options):


    (H, W) = image.shape[:2]

    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    blob = cv2.dnn.blobFromImage(
        image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    
    layerOutputs = net.forward(ln)
    

    # show timing information on YOLO
    

    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > options['confidence']:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, options['confidence'], options['threshold'])
    detections=[]
    if len(idxs) > 0:
        for i in idxs.flatten():
            temp_list=[]
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])           

            temp_list.append(x)
            temp_list.append(y)
            temp_list.append(w)
            temp_list.append(h)
            temp_list.append(Labels[classIDs[i]])
            temp_list.append(confidences[i])
            temp_list.append(classIDs[i])
            
            
            
            detections.append(temp_list)
    
    return detections

In [4]:
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))




frame1=cv2.imread('frames/0.jpg')
detection_calls=0

def get_flow_vector_count_timer(new_image):
    
    global frame1
    old_gray    = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    
    p0          =  cv2.goodFeaturesToTrack(old_gray, 1000, 0.01, 10)
    
    frame_gray  =  cv2.cvtColor(new_image,cv2.COLOR_BGR2GRAY)

    
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)


    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    count=0
    
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        if abs(d-b)>1 or abs(a-c)>1:
            count = count +1     

    return count    

In [5]:
start = time.time()

for i in range(0,1083):
    new_image = cv2.imread('frames/{}.jpg'.format(i))

    count = get_flow_vector_count_timer(new_image)
    
    if count>28:
        
        detection_calls=detection_calls+1

        frame1 = new_image
        predict_object_image(new_image,net_default,LABELS_default,options_v3_default)
        
end = time.time()
print(end-start)
print(detection_calls)

326.17850375175476
609


In [43]:
# with open('dynaimic_array_liucas.data', 'wb') as f:
#     pickle.dump(dp, f)

In [24]:
dp = pickle.load( open( "dynaimic_array_liucas.data", "rb" ) )

In [ ]:
if compared: 
    check_function()
    